In [1]:
import cProfile
import os
import statistics as st
import plotly.graph_objects as go
import random as rd
import time
import pandas as pd
import math
from collections import defaultdict
import numpy as np
import itertools as it
import Utilities.experiment_utils as eu
import unit_test as ut
import Games.mnk as mnk
import Games.Carcassonne.Carcassonne as carc
#import Games.carcassonne_older as csn
import Agents.random as arand
import Agents.vanilla_mcts as mcts 
import Agents.siea_mcts as siea_mcts
import Games.function_optimisation as fo
import Utilities.logs_management as lm
import Games.chess_64 as chess_64
#cProfile.run("wins =  random_games(10000, base_gs)")
#ut.run()

In [4]:
mcts_agent = mcts.MCTS_Player(max_iterations = 0)
siea_mcts_agent = siea_mcts.SIEA_MCTS_Player(max_iterations = 0)
#game_state = carc.CarcassonneState(initial_tile_quantities=[1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0])
#game_state = carc.CarcassonneState(initial_tile_quantities=[1 for _ in range(24)])
game_state = carc.CarcassonneState()
#game_state = mnk.GameState(3,3,3)
game_state.set_initial_state()
random_action = mcts_agent.choose_action(game_state)
print("Max score", game_state.max_possible_score)
print(random_action)
print(len(game_state.available_actions))
print(mcts_agent.view_mcts_tree())

Vanilla_MCTS : Random move returned
Max score 234
TileIndex: 10, (X,Y): (0,-1), Rotation: 0, Meeple Location: Farm, Location Index: 1
8

0:Decision_node, edge:None, visits:0, avg_reward:nan, children:0, ret:, tree_policy_formula:None


In [228]:
#mcts iteration
node = mcts_agent.root_node

#Selection
node = mcts_agent.selection(node) #this will always return a decision node

#Expansion
if node.can_be_expanded():
    node = mcts_agent.expansion(node)

#Simulation
reward = mcts_agent.simulation(node, mcts_agent.rollouts, mcts_agent.default_policy) 

#Backpropagation
mcts_agent.backpropagation(node, reward)

print(mcts_agent.view_mcts_tree())


0:Decision_node, edge:None, visits:226, avg_reward:0.294, children:2, ret:, tree_policy_formula:None
--1:Chance_node, edge:TileIndex: 0, (X,Y): (0,1), Rotation: 0, visits:22, avg_reward:-0.136, children:1, ret:Draw random tile, tree_policy_formula:0.566
----2:Decision_node, edge:RandomEvent id:13, prob:1 type:Draw random tile, visits:22, avg_reward:-0.136, children:18, ret:, tree_policy_formula:0.666
------6:Decision_node, edge:TileIndex: 13, (X,Y): (0,-1), Rotation: 270, visits:1, avg_reward:0, children:0, ret:, tree_policy_formula:2.49
------9:Decision_node, edge:TileIndex: 13, (X,Y): (0,-1), Rotation: 270, Meeple Location: City, Location Index: 0, visits:1, avg_reward:-0.111, children:0, ret:, tree_policy_formula:2.6
------12:Decision_node, edge:TileIndex: 13, (X,Y): (0,2), Rotation: 180, Meeple Location: Farm, Location Index: 0, visits:1, avg_reward:0, children:0, ret:, tree_policy_formula:2.49
------16:Decision_node, edge:TileIndex: 13, (X,Y): (1,1), Rotation: 270, visits:1, avg_

In [5]:
TotalGames = 10
logs = True
random_seed = 1
if random_seed is not None: 
    rd.seed(random_seed)
    np.random.seed(random_seed)
else: 
    random_seed = rd.randint(0, 2**32)
    print("meh")
    rd.seed(random_seed)
    np.random.seed(random_seed)

game_state = carc.CarcassonneState(initial_tile_quantities=[1,2,1,2,1,2,1,2,1,2,1,2,1,2,1,2,1,0,0,0,0,0,0,0])
#game_state = carc.CarcassonneState(initial_tile_quantities=[1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0])
#game_state = mnk.GameState(m=3,n=3,k=3)
game_state.set_initial_state()
agent3 = arand.RandomPlayer()
agent2 = mcts.MCTS_Player(max_iterations=100)
players = [agent2, agent3]
games_count = 0
WinnerCount = {0:0, 1:0, "Draw":0}

all_action_logs = pd.DataFrame()
all_game_logs = pd.DataFrame()

ST = time.time()

for game in range(TotalGames):
    #gs = game_state.duplicate()
    gs = game_state.duplicate()

    #Set logs
    action_logs = pd.DataFrame()
    game_logs = pd.DataFrame()
    if logs:
        for p in players:
            p.logs = True

    #Play game
    safe_count = 0
    while not gs.is_terminal:
        start_time = time.time()
        action = players[gs.player_turn].choose_action(gs)
        #action = rd.choice(gs.available_actions)
        selection_time = time.time() - start_time

        #Update logs
        if logs:
            action_log = players[gs.player_turn].choose_action_logs #Assumes this log is single row
            action_log["game_index"] = games_count
            action_log["selection_time"] = selection_time
            action_log["returned_action"] = str(action)
            action_log["pg_player"] = str(players[gs.player_turn])
            action_logs = pd.concat([action_logs, action_log], ignore_index=True)
            game_logs = pd.concat([game_logs, gs.logs_data()], ignore_index=True)

        #safety check      
        safe_count += 1
        if safe_count > 1000: 
            print("Safe count exceeded")
            print(gs.logs_data())
            print("Last action:" + str(action))
            break
        assert safe_count < 1000, "Safe count exceeded"

        #Make action
        gs.make_action(action)

    if logs:
        #Final logs by action
        final_logs_by_action = pd.concat([action_logs, game_logs], axis=1)


        #Final logs by game
        final_logs_by_game_dict = {}
        for i, player in enumerate(players):
            final_logs_by_game_dict["Player_" + str(i)] = str(player)
        final_logs_by_game_dict["game_random_seed"] = random_seed
        final_logs_by_game_dict["game_index"] = games_count
        final_logs_by_game = pd.DataFrame(final_logs_by_game_dict, index=[0])
        final_logs_by_game = pd.concat([final_logs_by_game, gs.logs_data()], axis=1)
            
    games_count += 1
    if gs.winner is None:
        WinnerCount["Draw"] += 1
    else: WinnerCount[gs.winner] += 1
    print("Game", games_count, "ended, winner:", gs.winner, "scores:", gs.Scores)

    all_action_logs = pd.concat([all_action_logs, final_logs_by_action], ignore_index=True)
    all_game_logs = pd.concat([all_game_logs, final_logs_by_game], ignore_index=True)

ET = time.time()

all_action_logs.to_csv(os.path.join("Outputs","test_carc","by_action.csv"))
all_game_logs.to_csv(os.path.join("Outputs","test_carc","by_game.csv"))
print("Games ended, time:", str(ET - ST))
print("WinnerCount",WinnerCount)

In [2]:
#Evolved formulas analysis, now in experiment
logs_path = os.path.join("Outputs","FO_single_decision_new")
data = pd.read_csv(os.path.join(logs_path, "logs_by_run.csv"))
evolved_formula_data = pd.DataFrame()
for agent in data["Player"].unique():
    if "EA" in agent:
        for f_index in data["Function_index"].unique():
            tdata = data[(data["Player"]==agent) & (data["Function_index"]==f_index)]
            fa_data = eu.evolved_formula_analysis(tdata)
            fa_data["Player"] = [agent]
            fa_data["Function_index"] = [f_index]
            evolved_formula_data = pd.concat([evolved_formula_data, fa_data])
evolved_formula_data.to_csv(os.path.join(logs_path, "evolved_formula_analysis.csv"), mode='a')

In [8]:
for file_name in ["evolution_logs.csv"]:
    experiment_path = os.path.join("Outputs","FO_single_decision_new2")
    file_path_list = lm.find_log_files(file_name, experiment_path)
    lm.combine_logs(experiment_path, file_name, file_path_list)

In [ ]:
#Plot histograms

n_bins = 2**7
experiment_path = os.path.join("Outputs","FO_single_decision_new2")
file_path_list = lm.find_log_files("tree_data.csv", experiment_path)
print(["\n" + f for f in file_path_list])
for function_index in [1]:#range(5):
  data_list = []
  subplot_titles = []
  file_paths = []

  #Add ea vanilla
  paths_to_remove = []
  for file_path in [f for f in file_path_list if "Function_" + str(function_index) in f]:
      agent_name = file_path.split(os.sep)[-2]
      if "_c" in agent_name:
        #data_list.append(pd.read_csv(file_path))
        if "1_4142" in agent_name:
           agent_name= agent_name.replace("1_4142", "\u221A\u03052\u0305")
        if "0_5" in agent_name:
           agent_name= agent_name.replace("0_5", "0.5")
        agent_name = agent_name.replace("MCTS_", "MCTS ")
        agent_name = agent_name.replace(" c", " c = ")
        subplot_titles += [agent_name]
        file_paths += [file_path]
        paths_to_remove += [file_path]
  for path in paths_to_remove:
    file_path_list.remove(path)

  paths_to_remove = []
  for file_path in [f for f in file_path_list if "Function_" + str(function_index) in f]:
    agent_name = file_path.split(os.sep)[-2]
    if "EA_" in agent_name and not "SIEA_" in agent_name:
      data_list.append(pd.read_csv(file_path))
      agent_name= agent_name.replace("_its", " ")
      agent_name += " iterations"
      subplot_titles += [agent_name]
      file_paths += [file_path]
  for path in paths_to_remove:
    file_path_list.remove(path)

  for file_path in [f for f in file_path_list if "Function_" + str(function_index) in f]:
    agent_name = file_path.split(os.sep)[-2]
    if "SIEA_" in agent_name:
      #data_list.append(pd.read_csv(file_path))
      agent_name= agent_name.replace("_its", " ")
      agent_name += " iterations"
      subplot_titles += [agent_name]
      file_paths += [file_path]
    
  if subplot_titles != [] and data_list != []:
    fop = fo.GameState(function_index=function_index)

    #"""
    interesting_runs = [0,10,16,36,41,45,72,73,97]
    interesting_data = []
    interesting_titles = ["Tunnel","Even","Random", "Exploratory", "Opposite", "Emergent 1", "Emergent 2", "Emergent 3", "UCB1 c=0.25"]
    for j in range(10):
      new_data_list = []
      subplot_titles = []
      dat = data_list[1]
      for i in range(j*10,(j+1)*10):
        tdat = dat.loc[dat["run"] == i]
        new_data_list = new_data_list + [tdat]
        if i==40:
          my_dat = tdat
        if i in interesting_runs:
          interesting_data += [tdat]
        subplot_titles += ["Run " + str(i)]
      subdata_list = new_data_list
    #"""

    plot = eu.fo_tree_histogram_average(interesting_data, 
                        fop.function, 
                        "",#"Function"+str(function_index+1) , 
                        divisions = 3,
                        n_buckets = n_bins, 
                        subplot_titles = [""] + interesting_titles,#file_paths, 
                        max_x_location=fop.max_location[0],
                        y_ref_value=None)
    plot.write_image(os.path.join(experiment_path, "Interesting_runs.png"))#"F" + str(function_index+1) + "_results"+ str(n_bins) + '.png'))#, width=800, height=1000)
    plot.show()
    #0,5,

In [4]:
max_fm = np.inf
max_iterations = 200
rollouts = 1
games = 5
#game = chess_64.GameState()
game = mnk.GameState(m=3,n=3,k=3)
game.set_initial_state()
agent0 = mcts.MCTS_Player(max_fm=max_fm, max_iterations = max_iterations, rollouts = rollouts, logs=True)
agent1 = siea_mcts.SIEA_MCTS_Player(max_fm=max_fm, max_iterations = max_iterations, rollouts = rollouts, logs=True, es_semantics_l = 0.1, es_semantics_u = 0.5)
agent2 = siea_mcts.SIEA_MCTS_Player(max_fm=max_fm, max_iterations = max_iterations, rollouts = rollouts, logs=True, unpaired_evolution=True, name="SIEA_MCTS_unpaired")
agent3 = arand.RandomPlayer()
eu.play_match([agent0, agent3], game, games, file_path=os.path.join("Outputs","Test_match_mnk"), logs=True, logs_dispatch_after=2)
#gp1 = eu.GamePlayer(game.duplicate(), [agent0, agent3])
#gp1.play_game()

Playing game 0 of 5, agents: ['Vanilla_MCTS', 'Random']
Playing game 1 of 5, agents: ['Vanilla_MCTS', 'Random']
Playing game 2 of 5, agents: ['Vanilla_MCTS', 'Random']
Playing game 3 of 5, agents: ['Vanilla_MCTS', 'Random']
Playing game 4 of 5, agents: ['Vanilla_MCTS', 'Random']
Playing game 0 of 5, agents: ['Random', 'Vanilla_MCTS']
Playing game 1 of 5, agents: ['Random', 'Vanilla_MCTS']
Playing game 2 of 5, agents: ['Random', 'Vanilla_MCTS']
Playing game 3 of 5, agents: ['Random', 'Vanilla_MCTS']
Playing game 4 of 5, agents: ['Random', 'Vanilla_MCTS']


In [ ]:
import chess #caps are white
import random as rd
import Games.chess_64 as chess_64
agent0 = mcts.MCTS_Player(max_iterations = 5, rollouts = 1, logs=True)
#state1 = chess_64.GameState()
state1 = mnk.GameState(m=3,n=3,k=3)
state1.set_initial_state()
outcomes = {}
for i in range(1):
    state=state1.duplicate()
    while state.is_terminal == False:
        #state.make_action(rd.choice(state.available_actions))
        state.make_action(agent0.choose_action(state))
        #print(state.board)
        state.view_game_state()
    if state.winner not in outcomes.keys():
        outcomes[state.winner] = 1
    else:
        outcomes[state.winner] += 1
print(outcomes)
#state.board


In [1]:
class LudoArrow():
    def __init__(self,  end_index=None, player_house=None):
        self.player_house = player_house #The player whose house this is
        self.end_index = end_index

class LudoSquare():
    def __init__(self, index, is_safe=False, arrow=None, content=None, next_square_index=None, player_house=None, is_victory_path=False):
        self.index = index
        self.is_safe = is_safe
        self.content = content #A list of pieces
        self.next_square_index = next_square_index #The index of the next square
        self.player_house = player_house #The player whose house this sends to
        self.is_victory_path = is_victory_path #Whether this is a victory path square

class LudoPiece():
    def __init__(self, index, player):
        self.index = index
        self.player = player
        
class GameState():
    def __init__(self, n_players, pieces_per_player=4):
        self.players = ["red", "green", "blue", "yellow"]
        self.n_players = n_players
        self.pieces_per_player = pieces_per_player
    def set_initial_state(self):
        self.board = {}
        self.players_houses = {12:"red", 25:"green", 38:"blue", 51:"yellow"}
        self.players_starting_index = {"red":15, "green":28, "blue":41, "yellow":2}
        for i in range(52):
            if i in [2, 10, 15, 23, 28, 36, 41, 49]: #safe squares
                square = LudoSquare(i, is_safe=True)
            elif i in [6, 19, 32, 45]: #squares with immediate arrows
                arrow = LudoArrow(end_index=i+1)
                square = LudoSquare(i, arrow=arrow)
            elif i in [10, 23, 36, 49]: #squares with arrows to the player houses
                arrow = LudoArrow(end_index=53, player_house=i)
                square = LudoSquare(i, arrow=arrow)
            elif i in [13, 26, 39, 52]:
                square = LudoSquare(i, player_house=self.players_houses[i])





